In [1]:
 %tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
import numpy as np
from google.colab import drive
from datetime import datetime
import ast
import time

try:
  import tensorflow.compat.v2 as tf
except Exception:
  pass

sample_rate = 100000
signals_count = 2
capture_size = 2048
train_size = 800
test_size = 200

dir_base_model = "/content/gdrive/My Drive/DeepLearning/STRIP/"
model_file = "/content/gdrive/My Drive/DeepLearning/STRIP/noise-vs-fm.model"
model_weights = "/content/gdrive/My Drive/DeepLearning/STRIP/checkpoints/cp-0006.ckpt"

print(tf.__version__)

2.2.0-rc3


In [2]:
drive.mount('/content/gdrive')

# Create a dtype with the binary data format and the desired column names
dt = np.dtype([('i', 'f4'), ('q', 'f4')])
# Load noisy data
data = np.fromfile(dir_base_model + "test-nothing.iq", dtype=dt)
dfa = pd.DataFrame(data)
dfa["label"] = "BLANK"
npArray = np.array(range(len(dfa)), dtype='f4', copy=True, order='K', subok=False, ndmin=0)
dfa["time"] = npArray / sample_rate
# Load FM data
data = np.fromfile(dir_base_model + "test-999.iq", dtype=dt)
dfb = pd.DataFrame(data)
dfb["label"] = "FM"
npArray = np.array(range(len(dfb)), dtype='f4', copy=True, order='K', subok=False, ndmin=0)
dfb["time"] = npArray / sample_rate

frames = [dfa, dfb]
result = pd.concat(frames)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train = []
test = []

# Build datasets for train 
for i in (0, train_size):
  ii = i * capture_size
  train.append(dfa[ii:ii + capture_size].values)
for i in (0, train_size):
  ii = i * capture_size
  train.append(dfb[ii:ii + capture_size].values)

# Build datasets for test 
for i in (0, test_size):
  ii = i * capture_size
  test.append(dfa[ii:ii + capture_size].values)
for i in (0, test_size):
  ii = i * capture_size
  test.append(dfb[ii:ii + capture_size].values)

In [4]:
# Plot some sample data to check iq files TRAIN
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly
fig = make_subplots(rows=1, cols=2)
font_options = dict(family="Courier New, monospace", size=18, color="#7f7f7f")
fig.add_trace(go.Scatter(x=dfa[1:capture_size]['time'], y=dfa[1:capture_size]['i'], mode='lines', name='i'), row=1, col=1)
fig.add_trace(go.Scatter(x=dfa[1:capture_size]['time'], y=dfa[1:capture_size]['q'], mode='lines', name='q'), row=1, col=1)
fig.update_layout(height=400, width=800, title="NOISE", xaxis_title="time", yaxis_title="value", font=font_options)
fig.add_trace(go.Scatter(x=dfb[1:capture_size]['time'], y=dfb[1:capture_size]['i'], mode='lines', name='i'), row=1, col=2)
fig.add_trace(go.Scatter(x=dfb[1:capture_size]['time'], y=dfb[1:capture_size]['q'], mode='lines', name='q'), row=1, col=2)
fig.update_layout(height=400, width=800, title="FM", xaxis_title="time", yaxis_title="value", font=font_options)
fig.show()

In [0]:
def get_signal_dataframes(df):
  interleave = 2
  dx = []
  dy = []
  tx = []
  ty = []

  # Generate train dataset 
  dfax = df[['i', 'q']] 
  dfay = df[['label']] 
  dfax = dfax * 10000.0
  for i in range(0, train_size):
    newx = dfax[interleave * i:interleave * i + capture_size]
    dx.append(newx)    
    newy = dfay[interleave * i:interleave * i + capture_size]
    dy.append(newy)

  # Offset to separate train from test datain df
  offset = train_size * interleave + capture_size

  # Generate test dataset 
  for i in range(0, test_size):
    newx = dfax[interleave * i + offset:interleave * i + capture_size + offset]
    tx.append(newx)
    newy = dfay[interleave * i + offset:interleave * i + capture_size + offset]
    ty.append(newy)

  # Convert lists to dataframes
  train_x = pd.DataFrame(dx) 
  train_y = pd.DataFrame(dy) 
  test_x = pd.DataFrame(tx) 
  test_y = pd.DataFrame(ty) 
  return train_x, train_y, test_x, test_y

def add_dataframe_for_signal(df, train_x, train_y, test_x, test_y):
  x, y, tx, ty = get_signal_dataframes(df)
  train_x.append(x)
  train_y.append(y)
  test_x.append(tx)
  test_y.append(ty)

train_x = []
train_y = []
test_x = []
test_y = []
add_dataframe_for_signal(dfa, train_x, train_y, test_x, test_y)
add_dataframe_for_signal(dfb, train_x, train_y, test_x, test_y)

# Build dataframes 
train_x = pd.concat(train_x)
train_y = pd.concat(train_y)
test_x = pd.concat(test_x)
test_y = pd.concat(test_y)

# Shuffle data
permutations = np.random.permutation(len(train_y))
train_x = train_x.iloc[permutations]
train_y = train_y.iloc[permutations]
permutations = np.random.permutation(len(test_x))
test_x = test_x.iloc[permutations]
test_y = test_y.iloc[permutations]

In [0]:
def create_np_array_x(d, size):
  new = np.ndarray(shape=(signals_count * size, 2, capture_size), dtype=float)
  # train or test size i
  for i in range(len(d.values)):
    # 2048 j
    for j in range(len((d.values)[i][0])): 
        new[i][0][j] = ((d.values)[i])[0].values[j][0]
        new[i][1][j] = ((d.values)[i])[0].values[j][1]
  return new

def create_np_array_y(d, size):
  new = np.ndarray(shape=(signals_count * size), dtype=np.int16)
  # train or test size i
  for i in range(len(d.values)):
    result = ((d.values)[i])[0].values[0][0]
    if (result == 'FM'):
      new[i] = 1
    else:
      new[i] = 0
  return new

train_x = create_np_array_x(train_x, train_size)
train_y = create_np_array_y(train_y, train_size)
test_x = create_np_array_x(test_x, test_size)
test_y = create_np_array_y(test_y, test_size)

In [0]:
# Prepare one line of data to predict
element = 0
samples_to_show = 5

td = np.ndarray(shape=(samples_to_show, 2, capture_size), dtype=float)
ld = np.ndarray(shape=(samples_to_show), dtype=int)

for j in range(samples_to_show):
  for i in range(capture_size):
    td[j][0][i] = test_x[element+j][0][i]
    td[j][1][i] = test_x[element+j][1][i]
  ld[j] = test_y[element+j]

def label_from_column(max_column):
  if (max_column==1):
    return "FM"
  if (max_column==0):
    return "Noise"
  return "Error"

In [0]:
def load_last_model1():
  model = tf.keras.models.load_model(model_file)
  model.load_weights(model_weights)
  return model

In [9]:
model = load_last_model1()
predictions = model.predict(td)

# Print predicted values
print(predictions)

# Build predicted values
listx = []
for i in range(len(predictions)):
  result = np.argmax(predictions[i], axis=None) 
  print("predicted " + str(result))
  listx.append(label_from_column(result))
print(listx)

# Print original values
print(ld)

[[[0.63561004 0.36438996]]

 [[0.6145806  0.3854195 ]]

 [[0.45510557 0.5448944 ]]

 [[0.5669895  0.4330105 ]]

 [[0.5769597  0.42304027]]]
predicted 0
predicted 0
predicted 1
predicted 0
predicted 0
['Noise', 'Noise', 'FM', 'Noise', 'Noise']
[0 0 1 0 0]


In [10]:
model.weights

[<tf.Variable 'conv1d/kernel:0' shape=(16, 2048, 10) dtype=float32, numpy=
 array([[[-9.82056092e-03, -1.20364549e-02,  7.36596249e-03, ...,
          -1.00708650e-02,  1.21320300e-02, -9.45941173e-03],
         [ 6.87265396e-03, -7.65357027e-03,  8.53036717e-03, ...,
          -3.51903215e-03,  1.11963414e-03, -5.56266028e-03],
         [-3.77604645e-03,  1.17288418e-02,  1.32650621e-02, ...,
           2.80192308e-03,  4.89156693e-04,  8.58671963e-03],
         ...,
         [-7.53284665e-03, -1.21260863e-02, -1.11955907e-02, ...,
          -7.61177996e-03,  2.79521942e-03,  7.67741539e-03],
         [-7.23281922e-03,  1.08904149e-02, -3.48022487e-03, ...,
           1.20710749e-02, -1.24100801e-02,  1.02532264e-02],
         [ 8.66528973e-03,  6.36776350e-03,  5.36138192e-03, ...,
          -4.13946249e-03,  1.05870888e-02,  2.02257745e-03]],
 
        [[-3.63315176e-03, -1.12806447e-04, -3.94309498e-03, ...,
          -4.68792301e-03,  1.15131866e-03,  5.39266504e-04],
         [-1

In [11]:
def test_function_printed(df, label, i, j, fig):
  #fig = go.Figure()
  fig.add_trace(go.Scatter(x=dfb[1:capture_size]['time'], y=df[0], mode='lines', name='i'), row=i, col=j)
  fig.add_trace(go.Scatter(x=dfb[1:capture_size]['time'], y=df[1], mode='lines', name='q'), row=i, col=j)
  
 
fig = make_subplots(rows=2, cols=3)
i = 0
test_function_printed(td[i], ld[i], 1, 1, fig)
i = 1
test_function_printed(td[i], ld[i], 1, 2, fig)
i = 2
test_function_printed(td[i], ld[i], 1, 3, fig)
i = 3
test_function_printed(td[i], ld[i], 2, 1, fig)
i = 4
test_function_printed(td[i], ld[i], 2, 2, fig)

fig.update_layout(
      title="This should be " + str(ld[0]) + ', ' + str(ld[1]) + ', ' + str(ld[2]) + ', ' + str(ld[3]) + ', ' + str(ld[4]),
      xaxis_title="time", yaxis_title="value", height=400, width=600, font=dict(family="Courier New, monospace", size=18, color="#7f7f7f")
  )
  
fig.show()

In [12]:
dfb[0:capture_size]['time']*10000.0

0         0.0
1         0.1
2         0.2
3         0.3
4         0.4
        ...  
2043    204.3
2044    204.4
2045    204.5
2046    204.6
2047    204.7
Name: time, Length: 2048, dtype: float64